## Intro
This notebook will:
#### extrac the embeddings for each model (baseline vs fine tuned) 
#### train a shallow model (xgboost ) 
#### test the validation set accuracy against the shallow model results 


In [30]:
import pathlib
import torch

from esm import FastaBatchedDataset, pretrained

In [ ]:
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")

def extract_embeddings(model, fasta_file,caslabel, output_dir, tokens_per_batch=4096, seq_length=1022,repr_layers=[33]):
    from transformers import EsmTokenizer, EsmModel
    import torch
    # read the fasta file and extract sequences

    
    seqs =[seq]
    inputs = tokenizer(seqs, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    x = last_hidden_states.detach()
    x =x.mean(axis=1)

In [37]:
def extract_embeddingsx(model_name, fasta_file,caslabel, output_dir, tokens_per_batch=4096, seq_length=1022,repr_layers=[33]):
    
    model, alphabet = pretrained.load_model_and_alphabet(model_name)
    model.eval()
    

    if torch.cuda.is_available():
        model = model.cuda('cuda:1')
    
#     caslist = ['cas1','cas2','cas3','cas4','cas10','cas12','cas14','cas9','cas13a', 'cas13b', 'cas13c','cas13d']
    dataset = FastaBatchedDataset.from_file(fasta_file)
    batches = dataset.get_batch_indices(tokens_per_batch, extra_toks_per_seq=1)

    data_loader = torch.utils.data.DataLoader(
        dataset, 
        collate_fn=alphabet.get_batch_converter(seq_length), 
        batch_sampler=batches
    )

    output_dir.mkdir(parents=True, exist_ok=True)
    
    with torch.no_grad():
        for batch_idx, (labels, strs, toks) in enumerate(data_loader):

            print(f'Processing batch {batch_idx + 1} of {len(batches)}')

            if torch.cuda.is_available():
                toks = toks.to(device="cuda:1", non_blocking=True)

            out = model(toks, repr_layers=repr_layers, return_contacts=False)
            

            logits = out["logits"].to(device="cpu")
            representations = {layer: t.to(device="cpu") for layer, t in out["representations"].items()}
            
            for i, label in enumerate(labels):
                entry_id = label.split()[0]
                
                filename = output_dir / f"{entry_id}.pt"
                truncate_len = min(seq_length, len(strs[i]))

                result = {"entry_id": entry_id}
                caslabel = ""
                for word in caslist:
                    if word.lower() in label.lower():
                        caslabel = word.lower()
                    
                result['label'] = caslabel
                result["mean_representations"] = {
                        layer: t[i, 1 : truncate_len + 1].mean(0).clone()
                        for layer, t in representations.items()
                    }
                
                
                
                torch.save(result, filename)
#                 if i >1:
#                     return(result)
                


In [38]:
def extract_embeddings0(model_name, fasta_file, caslabel, output_dir, tokens_per_batch=4096, seq_length=1022, repr_layers=[33]):
    from Bio import SeqIO
    import torch
    from torch.utils.data import DataLoader
    
    # Load your model and alphabet (assuming a custom function or similar exists)
    model, alphabet = pretrained.load_model_and_alphabet(model_name)
    model.eval()

    # Setup model for multiple GPUs
    if torch.cuda.is_available():
        model = model.cuda()
        model = torch.nn.DataParallel(model)

    # Load dataset
    dataset = FastaBatchedDataset.from_file(fasta_file)
    batches = dataset.get_batch_indices(tokens_per_batch, extra_toks_per_seq=1)

    # Prepare DataLoader
    data_loader = DataLoader(
        dataset, 
        collate_fn=alphabet.get_batch_converter(seq_length), 
        batch_sampler=batches
    )

    output_dir.mkdir(parents=True, exist_ok=True)
    
    with torch.no_grad():
        for batch_idx, (labels, strs, toks) in enumerate(data_loader):
            print(f'Processing batch {batch_idx + 1} of {len(batches)}')

            toks = toks.cuda()  # Ensure tokens are on GPU
            out = model(toks, repr_layers=repr_layers, return_contacts=False)
            
            # Process outputs
            logits = out["logits"].cpu()
            representations = {layer: t.cpu() for layer, t in out["representations"].items()}

            for i, label in enumerate(labels):
                entry_id = label.split()[0]
                filename = output_dir / f"{entry_id}.pt"
                truncate_len = min(seq_length, len(strs[i]))

                result = {"entry_id": entry_id, "label": caslabel}
                result["mean_representations"] = {
                        layer: t[i, 1 : truncate_len + 1].mean(0).clone()
                        for layer, t in representations.items()
                    }
                
                
                torch.save(result, filename)
                
                


In [39]:
caslist = ['cas1','cas2','cas3','cas4','cas5','cas6','cas7','cas8','cas9','cas10','cas11','cas12','cas13']


In [40]:
# model_name = 'esm2_t33_650M_UR50D'
model_name = 'esm2_t6_8M_UR50D'


In [41]:
torch.cuda.empty_cache()


In [42]:
rep_layer_number = 6 # for the 8 m model, for 650 use 33 

for cas in caslist: 
    
    casfolder = f"/home/salaris/protein_model/data/{cas}/"
    
    training_fasta_file = pathlib.Path(casfolder + cas + '_training.fasta')
    validation_fasta_file = pathlib.Path(casfolder + cas + '_validation.fasta')
    
    training_embedding_folder = pathlib.Path(casfolder  + "_" +model_name + "_" + 'embeddings/' +  'training/')
    validation_embedding_folder = pathlib.Path(casfolder  + "_" +model_name + "_" + 'embeddings/' +   'validation/')
    print(training_embedding_folder, validation_embedding_folder)
    print(training_fasta_file, validation_fasta_file)
    extract_embeddings(model_name, 
                       fasta_file= training_fasta_file,caslabel= cas, 
                       output_dir= training_embedding_folder, tokens_per_batch=2048 * 2, seq_length=1022,repr_layers=[rep_layer_number])

    extract_embeddings(model_name, 
                       fasta_file= validation_fasta_file,caslabel= cas, 
                       output_dir= validation_embedding_folder, tokens_per_batch=2048 * 2 , seq_length=1022,repr_layers=[rep_layer_number])
    

/home/salaris/protein_model/data/cas1/_esm2_t6_8M_UR50D_embeddings/training /home/salaris/protein_model/data/cas1/_esm2_t6_8M_UR50D_embeddings/validation
/home/salaris/protein_model/data/cas1/cas1_training.fasta /home/salaris/protein_model/data/cas1/cas1_validation.fasta
Processing batch 1 of 1116
Processing batch 2 of 1116
Processing batch 3 of 1116
Processing batch 4 of 1116
Processing batch 5 of 1116
Processing batch 6 of 1116
Processing batch 7 of 1116
Processing batch 8 of 1116
Processing batch 9 of 1116
Processing batch 10 of 1116
Processing batch 11 of 1116
Processing batch 12 of 1116
Processing batch 13 of 1116
Processing batch 14 of 1116
Processing batch 15 of 1116
Processing batch 16 of 1116
Processing batch 17 of 1116
Processing batch 18 of 1116
Processing batch 19 of 1116
Processing batch 20 of 1116
Processing batch 21 of 1116
Processing batch 22 of 1116
Processing batch 23 of 1116
Processing batch 24 of 1116
Processing batch 25 of 1116
Processing batch 26 of 1116
Processin

Processing batch 278 of 1116
Processing batch 279 of 1116
Processing batch 280 of 1116
Processing batch 281 of 1116
Processing batch 282 of 1116
Processing batch 283 of 1116
Processing batch 284 of 1116
Processing batch 285 of 1116
Processing batch 286 of 1116
Processing batch 287 of 1116
Processing batch 288 of 1116
Processing batch 289 of 1116
Processing batch 290 of 1116
Processing batch 291 of 1116
Processing batch 292 of 1116
Processing batch 293 of 1116
Processing batch 294 of 1116
Processing batch 295 of 1116
Processing batch 296 of 1116
Processing batch 297 of 1116
Processing batch 298 of 1116
Processing batch 299 of 1116
Processing batch 300 of 1116
Processing batch 301 of 1116
Processing batch 302 of 1116
Processing batch 303 of 1116
Processing batch 304 of 1116
Processing batch 305 of 1116
Processing batch 306 of 1116
Processing batch 307 of 1116
Processing batch 308 of 1116
Processing batch 309 of 1116
Processing batch 310 of 1116
Processing batch 311 of 1116
Processing bat

Processing batch 562 of 1116
Processing batch 563 of 1116
Processing batch 564 of 1116
Processing batch 565 of 1116
Processing batch 566 of 1116
Processing batch 567 of 1116
Processing batch 568 of 1116
Processing batch 569 of 1116
Processing batch 570 of 1116
Processing batch 571 of 1116
Processing batch 572 of 1116
Processing batch 573 of 1116
Processing batch 574 of 1116
Processing batch 575 of 1116
Processing batch 576 of 1116
Processing batch 577 of 1116
Processing batch 578 of 1116
Processing batch 579 of 1116
Processing batch 580 of 1116
Processing batch 581 of 1116
Processing batch 582 of 1116
Processing batch 583 of 1116
Processing batch 584 of 1116
Processing batch 585 of 1116
Processing batch 586 of 1116
Processing batch 587 of 1116
Processing batch 588 of 1116
Processing batch 589 of 1116
Processing batch 590 of 1116
Processing batch 591 of 1116
Processing batch 592 of 1116
Processing batch 593 of 1116
Processing batch 594 of 1116
Processing batch 595 of 1116
Processing bat

Processing batch 847 of 1116
Processing batch 848 of 1116
Processing batch 849 of 1116
Processing batch 850 of 1116
Processing batch 851 of 1116
Processing batch 852 of 1116
Processing batch 853 of 1116
Processing batch 854 of 1116
Processing batch 855 of 1116
Processing batch 856 of 1116
Processing batch 857 of 1116
Processing batch 858 of 1116
Processing batch 859 of 1116
Processing batch 860 of 1116
Processing batch 861 of 1116
Processing batch 862 of 1116
Processing batch 863 of 1116
Processing batch 864 of 1116
Processing batch 865 of 1116
Processing batch 866 of 1116
Processing batch 867 of 1116
Processing batch 868 of 1116
Processing batch 869 of 1116
Processing batch 870 of 1116
Processing batch 871 of 1116
Processing batch 872 of 1116
Processing batch 873 of 1116
Processing batch 874 of 1116
Processing batch 875 of 1116
Processing batch 876 of 1116
Processing batch 877 of 1116
Processing batch 878 of 1116
Processing batch 879 of 1116
Processing batch 880 of 1116
Processing bat

Processing batch 11 of 126
Processing batch 12 of 126
Processing batch 13 of 126
Processing batch 14 of 126
Processing batch 15 of 126
Processing batch 16 of 126
Processing batch 17 of 126
Processing batch 18 of 126
Processing batch 19 of 126
Processing batch 20 of 126
Processing batch 21 of 126
Processing batch 22 of 126
Processing batch 23 of 126
Processing batch 24 of 126
Processing batch 25 of 126
Processing batch 26 of 126
Processing batch 27 of 126
Processing batch 28 of 126
Processing batch 29 of 126
Processing batch 30 of 126
Processing batch 31 of 126
Processing batch 32 of 126
Processing batch 33 of 126
Processing batch 34 of 126
Processing batch 35 of 126
Processing batch 36 of 126
Processing batch 37 of 126
Processing batch 38 of 126
Processing batch 39 of 126
Processing batch 40 of 126
Processing batch 41 of 126
Processing batch 42 of 126
Processing batch 43 of 126
Processing batch 44 of 126
Processing batch 45 of 126
Processing batch 46 of 126
Processing batch 47 of 126
P

...

Processing batch 712 of 1491
Processing batch 713 of 1491
Processing batch 714 of 1491
Processing batch 715 of 1491
Processing batch 716 of 1491
Processing batch 717 of 1491
Processing batch 718 of 1491
Processing batch 719 of 1491
Processing batch 720 of 1491
Processing batch 721 of 1491
Processing batch 722 of 1491
Processing batch 723 of 1491
Processing batch 724 of 1491
Processing batch 725 of 1491
Processing batch 726 of 1491
Processing batch 727 of 1491
Processing batch 728 of 1491
Processing batch 729 of 1491
Processing batch 730 of 1491
Processing batch 731 of 1491
Processing batch 732 of 1491
Processing batch 733 of 1491
Processing batch 734 of 1491
Processing batch 735 of 1491
Processing batch 736 of 1491
Processing batch 737 of 1491
Processing batch 738 of 1491
Processing batch 739 of 1491
Processing batch 740 of 1491
Processing batch 741 of 1491
Processing batch 742 of 1491
Processing batch 743 of 1491
Processing batch 744 of 1491
Processing batch 745 of 1491
Processing bat

Processing batch 995 of 1491
Processing batch 996 of 1491
Processing batch 997 of 1491
Processing batch 998 of 1491
Processing batch 999 of 1491
Processing batch 1000 of 1491
Processing batch 1001 of 1491
Processing batch 1002 of 1491
Processing batch 1003 of 1491
Processing batch 1004 of 1491
Processing batch 1005 of 1491
Processing batch 1006 of 1491
Processing batch 1007 of 1491
Processing batch 1008 of 1491
Processing batch 1009 of 1491
Processing batch 1010 of 1491
Processing batch 1011 of 1491
Processing batch 1012 of 1491
Processing batch 1013 of 1491
Processing batch 1014 of 1491
Processing batch 1015 of 1491
Processing batch 1016 of 1491
Processing batch 1017 of 1491
Processing batch 1018 of 1491
Processing batch 1019 of 1491
Processing batch 1020 of 1491
Processing batch 1021 of 1491
Processing batch 1022 of 1491
Processing batch 1023 of 1491
Processing batch 1024 of 1491
Processing batch 1025 of 1491
Processing batch 1026 of 1491
Processing batch 1027 of 1491
Processing batc

Processing batch 1271 of 1491
Processing batch 1272 of 1491
Processing batch 1273 of 1491
Processing batch 1274 of 1491
Processing batch 1275 of 1491
Processing batch 1276 of 1491
Processing batch 1277 of 1491
Processing batch 1278 of 1491
Processing batch 1279 of 1491
Processing batch 1280 of 1491
Processing batch 1281 of 1491
Processing batch 1282 of 1491
Processing batch 1283 of 1491
Processing batch 1284 of 1491
Processing batch 1285 of 1491
Processing batch 1286 of 1491
Processing batch 1287 of 1491
Processing batch 1288 of 1491
Processing batch 1289 of 1491
Processing batch 1290 of 1491
Processing batch 1291 of 1491
Processing batch 1292 of 1491
Processing batch 1293 of 1491
Processing batch 1294 of 1491
Processing batch 1295 of 1491
Processing batch 1296 of 1491
Processing batch 1297 of 1491
Processing batch 1298 of 1491
Processing batch 1299 of 1491
Processing batch 1300 of 1491
Processing batch 1301 of 1491
Processing batch 1302 of 1491
Processing batch 1303 of 1491
Processing

Processing batch 61 of 169
Processing batch 62 of 169
Processing batch 63 of 169
Processing batch 64 of 169
Processing batch 65 of 169
Processing batch 66 of 169
Processing batch 67 of 169
Processing batch 68 of 169
Processing batch 69 of 169
Processing batch 70 of 169
Processing batch 71 of 169
Processing batch 72 of 169
Processing batch 73 of 169
Processing batch 74 of 169
Processing batch 75 of 169
Processing batch 76 of 169
Processing batch 77 of 169
Processing batch 78 of 169
Processing batch 79 of 169
Processing batch 80 of 169
Processing batch 81 of 169
Processing batch 82 of 169
Processing batch 83 of 169
Processing batch 84 of 169
Processing batch 85 of 169
Processing batch 86 of 169
Processing batch 87 of 169
Processing batch 88 of 169
Processing batch 89 of 169
Processing batch 90 of 169
Processing batch 91 of 169
Processing batch 92 of 169
Processing batch 93 of 169
Processing batch 94 of 169
Processing batch 95 of 169
Processing batch 96 of 169
Processing batch 97 of 169
P

Processing batch 85 of 213
Processing batch 86 of 213
Processing batch 87 of 213
Processing batch 88 of 213
Processing batch 89 of 213
Processing batch 90 of 213
Processing batch 91 of 213
Processing batch 92 of 213
Processing batch 93 of 213
Processing batch 94 of 213
Processing batch 95 of 213
Processing batch 96 of 213
Processing batch 97 of 213
Processing batch 98 of 213
Processing batch 99 of 213
Processing batch 100 of 213
Processing batch 101 of 213
Processing batch 102 of 213
Processing batch 103 of 213
Processing batch 104 of 213
Processing batch 105 of 213
Processing batch 106 of 213
Processing batch 107 of 213
Processing batch 108 of 213
Processing batch 109 of 213
Processing batch 110 of 213
Processing batch 111 of 213
Processing batch 112 of 213
Processing batch 113 of 213
Processing batch 114 of 213
Processing batch 115 of 213
Processing batch 116 of 213
Processing batch 117 of 213
Processing batch 118 of 213
Processing batch 119 of 213
Processing batch 120 of 213
Process